# Epidemiology of Sedation in Mechanical Ventilation

In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/wliao0504/code/clif/CLIF-epi-of-sedation'

In [13]:
import pandas as pd
import duckdb
from utils import pyCLIF as pc
from utils.waterfall import process_resp_support_waterfall

## Cohort Identification

In [11]:
resp = pc.load_data("clif_respiratory_support")

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_respiratory_support.parquet


In [14]:
resp_f = process_resp_support_waterfall(resp)

✦ Phase 0: initialise & create hourly scaffold
FIO2_SET mean= 0.49026212251411716 is within the required range
Creating hourly scaffold for each encounter


Creating hourly scaffolds: 100%|██████████| 70103/70103 [00:03<00:00, 21747.59it/s]


✦ Phase 1: heuristic inference of device / mode
✦ Phase 2: build device / mode hierarchical IDs
✦ Phase 3: numeric down/up-fill inside mode_name_id blocks
Applying waterfall fill to 70103 encounters...


Waterfall fill by mode_name_id: 100%|██████████| 419988/419988 [02:32<00:00, 3607.29it/s]/Users/wliao0504/code/clif/CLIF-epi-of-sedation/.venv-sedation/lib/python3.10/site-packages/tqdm/std.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return getattr(df, df_function)(wrapper, **kwargs)
Waterfall fill by mode_name_id: 100%|██████████| 419988/419988 [02:49<00:00, 2473.83it/s]


✦ Phase 4: final deduplication & ordering
[OK] Respiratory-support waterfall complete.


In [52]:
query = """
SELECT
    hospitalization_id,
    date_trunc('hour', recorded_dttm) as date_hour,
    MAX(CASE WHEN device_category = 'imv' THEN 1 ELSE 0 END) as on_imv,
    SUM(on_imv) OVER (
        PARTITION BY hospitalization_id 
        ORDER BY date_hour 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
        as total_hrs_on_imv,
    ROW_NUMBER() OVER (PARTITION BY hospitalization_id ORDER BY date_hour) as hr_index,
    CASE WHEN (on_imv = 1) AND (hr_index = 24) AND (total_hrs_on_imv = 24) THEN 1 ELSE 0 END as hr_24_on_imv,
    CASE WHEN (on_imv = 1) AND (hr_index = 72) AND (total_hrs_on_imv = 72) THEN 1 ELSE 0 END as hr_72_on_imv,
FROM resp_f
WHERE hospitalization_id IN (21738444) -- focal cases
GROUP BY hospitalization_id, date_hour
ORDER BY hospitalization_id, date_hour
"""
df1 = duckdb.sql(query).to_df()

In [49]:
query = """
SELECT *
FROM df1
WHERE hospitalization_id IN (
    SELECT DISTINCT hospitalization_id
    FROM df1
    WHERE hr_24_on_imv = 1
)
"""
df2 = duckdb.sql(query).to_df()